<span style="font-size: 20px; font-weight: bold;">Importing necessary libraries and seeding for reproducibility of the solution</span>


In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import random
import copy
import time
import math
import scipy.special as sps
import plotly.express as px
import matplotlib.pyplot as plt
from pyomo.environ import *
from collections import OrderedDict
random.seed(10)
np.random.seed(10)

<span style="font-size: 20px; font-weight: bold;">Reading the respective input file from the Data Folder. Declare the number of clusters you would want to experiment with using the parameter "num_clusters". In the paper it is recommended to use num_clusters = 5.</span>

In [2]:
num_clusters = 5

def get_df(num_clusters):
    if num_clusters == 5:
        df = pd.read_excel("Indigo_Fleet.xlsx",index_col=False)
    if num_clusters == 3:
        df = pd.read_excel("Indigo_Fleet_cluster3.xlsx",index_col=False)
    if num_clusters == 7:
        df = pd.read_excel("Indigo_Fleet_cluster7.xlsx",index_col=False)
    if num_clusters == 9:
        df = pd.read_excel("Indigo_Fleet_cluster9.xlsx",index_col=False)
    return df

df = get_df(num_clusters)

<span style="font-size: 20px; font-weight: bold;">Creating the necessary Data Structure for Flight Leg.</span>

In [3]:
class airport:
    
    def __init__(self, aport, aircraftCount_dict):
        self.aport = aport
        self.aircraftCount_dict = aircraftCount_dict

In [4]:
class originAirport:
    
    def __init__(self, airport, deptime, depday):
        self.airport = airport
        self.deptime = deptime
        self.depday = depday

In [5]:
class destinationAirport:
    
    def __init__(self, airport, arrtime, arrday, elapsed_time, total_prop_delay, flight_time):
        self.airport = airport
        self.arrtime = arrtime
        self.arrday = arrday
        self.elapsed_time = elapsed_time
        self.total_prop_delay = total_prop_delay
        self.flight_time = flight_time

In [6]:
class QTable:
    
    def __init__(self, origin1, dest1, arrival1, departure1, origin2, dest2, arrival2, departure2, val
                 , totalPropDelay, slack_time):
        self.origin1 = origin1
        self.dest1 = dest1
        self.arrival1 = arrival1
        self.departure1 = departure1
        self.origin2 = origin2
        self.dest2 = dest2
        self.arrival2 = arrival2
        self.departure2 = departure2
        self.val = val
        self.totalPropDelay = totalPropDelay
        self.slack_time = slack_time
        
class Reward:
    
    def __init__(self, origin1, dest1, arrival1, departure1, origin2, dest2, arrival2, departure2, reward, number):
        self.origin1 = origin1
        self.dest1 = dest1
        self.arrival1 = arrival1
        self.departure1 = departure1
        self.origin2 = origin2
        self.dest2 = dest2
        self.arrival2 = arrival2
        self.departure2 = departure2
        self.reward = reward
        self.number = number

In [7]:
class FlightLeg:

    def __init__(self,origin,dest,flights,day,arr_delay,dep_datetime,arr_datetime,flighttime,
                 total_propagated_delay,pointer,elapsed_time, processed, firstnodeorigin, averageratings
                 , priorityIndex, priorityAssigned, cluster_id
                 , expected_demand , sum_prev_priority, per_seat
                 , best_combination_value, normalized_per_seat, fleet, pre_averageratings
                 , serviceratings = [], foodratings = [], entertainmentratings = [], comfortratings = []):
        self.origin = origin
        self.dest = dest
        self.flights = flights
        self.day = day
        self.arr_delay = arr_delay
        self.dep_datetime = self.dateTimeObject(str(dep_datetime))
        self.arr_datetime = self.dateTimeObject(str(arr_datetime))
        self.flighttime = flighttime
        self.total_propagated_delay = total_propagated_delay
        self.pointer = pointer
        self.elapsed_time = elapsed_time
        self.processed = processed
        self.firstnodeorigin = firstnodeorigin
        self.priorityIndex = priorityIndex
        self.priorityAssigned = priorityAssigned
        self.serviceratings = serviceratings
        self.foodratings = foodratings
        self.entertainmentratings = entertainmentratings
        self.comfortratings = comfortratings
        self.averageratings = averageratings
        self.cluster_id = cluster_id
        self.sum_prev_priority = sum_prev_priority
        self.best_combination_value = best_combination_value
        self.expected_demand = expected_demand
        self.per_seat = per_seat
        self.normalized_per_seat = normalized_per_seat
        self.fleet = fleet
        self.pre_averageratings = pre_averageratings
        
    def dateTimeObject(self,String):
        return datetime.datetime.strptime(String,"%Y-%m-%d %H:%M:%S")

<span style="font-size: 20px; font-weight: bold;">Functions for necessary calculations of slack time, propagated delays, time elapsed and $\xi$</span>

In [8]:
def calculate_slack_time_seconds(Dept,Arr, day1, day2, mean_turnaround_time):
    if day1 >= day2:
        return ((Dept-Arr).total_seconds() - mean_turnaround_time)
    else:
        return -99

def check_slack_time_seconds_validity(slack_time):
    return (24.*3600)>=slack_time>=0 #Check for flights within stipulated time for connection not beyond that

def check_time_elapsed_validity_seconds(time_elapsed):
    return (0<=time_elapsed<=36*60*60)

def calculate_propagated_delay_seconds(prev_prop_delay,arrival_delay,slack_time):
    # prop_delay = prev_prop_delay+arrival_delay*60-slack_time # when arrival delay is in minutes
    prop_delay = prev_prop_delay+arrival_delay-slack_time # when arrival delay is in seconds
    return max(0, prop_delay)

def calculate_time_elapsed(prev_elapsed_time,Arr,Dept):
    return prev_elapsed_time+(Dept-Arr).total_seconds()

def propDelaySeconds_RL(n1, n2):
    slack_time = calculate_slack_time_seconds(n2.dep_datetime, n1.arr_datetime, n1.day, n2.day, 0.0)
    prop_delay = calculate_propagated_delay_seconds(n1.total_propagated_delay, n2.arr_delay, slack_time)
    
    return prop_delay

<span style="font-size: 20px; font-weight: bold;">Integer Linear Program Formulation using pyomo. The function returns the selected string, flight legs selected to assign their respective $\xi$ values and list of propagated delays of the selected strings.</span>

In [9]:
#Optimization master problem
def route_optimizer(stagewise_routes, prop_delay, route_strings
                    , priority_list, priority_number, sflights, assignedFlights, itr, penaltyCost
                    , priorityWeight, delayWeight, uniquePriorities, reward_list):
    
    model = ConcreteModel()
    n = len(stagewise_routes)
    penaltyCost = 10000

    
    uniqueFlights = []
    uniquePriority = []
    fullPriority = []
    flightPriority = []
    multipleFlight = []
    fp_dict = {}

    for sr in stagewise_routes:
        for s in sr:
            uniqueFlights.append(s)
            
    for pl in priority_list:
        for p in pl:
            fullPriority.append(p)
            
    for f in uniqueFlights:
        if f not in fp_dict:
            fp_dict[f] = fullPriority[uniqueFlights.index(f)]
    
    uniqueFlights = set(uniqueFlights)
    
    for key, value in fp_dict.items():
        uniquePriority.append(value)
        
    m = len(sflights)
    nparray = []
    flightString = {}
    ff = 0
    for i in sflights:
        if ff not in flightString:
            flightString[ff] = []
        
        npList=[]
        cc = 0
        for j in stagewise_routes:
            if i in j:
                npList.append(1)
                flightString[ff].append(cc)
            else:
                npList.append(0)
            cc+=1
        nparray.append(npList)
        ff+=1
    varList = []
    fvarList = []
    stringFlight = {}
    flightString = {}
    
    for j in range(len(sflights)):
        fvarList.append(j)
        if j not in flightString:
            flightString[j] = []

    for i in range(n):
        if i not in stringFlight:
            stringFlight[i] = []
            
            for s in sflights:
                if s in stagewise_routes[i]:
                    stringFlight[i].append(sflights.index(s))
                    flightString[sflights.index(s)].append(i)
                    
    for i in range(n):
        varList.append(i)

    for s in sflights:
        mf = 0
        for key, value in stringFlight.items():
            if sflights.index(s) in value:
                mf+=1
        if mf > 1:
            multipleFlight.append(sflights.index(s))
        
    model.xVar = Var(varList, within = Integers, bounds = (0,1), initialize = 0)
    model.fVar = Var(fvarList, bounds = (0,1), initialize = 0)
    
    ft = 0
    if len(assignedFlights) > 0:
        model.assignedFlights_constraint = ConstraintList()
        for f in sflights:
            if f in assignedFlights:
                model.assignedFlights_constraint.add(model.fVar[ft] == 1)
            ft+=1
            
    model.numberFlights_constraint = Constraint(expr = sum(model.fVar[sflights.index(f)] for f in sflights
                                                           if f not in assignedFlights) <= priority_number)
    
    model.cover_constraint = ConstraintList()
    for i in range(m):
        a=nparray[i]
        add=0
        
        for j in range(n):
            k=a[j]*model.xVar[j]
            add=add+k
        
        model.cover_constraint.add(add <=1)

    model.airport_capacity_constraint = ConstraintList()
    model.airport_capacity_constraint.add(sum(model.xVar[i] for i in range(len(model.xVar))) <= len(model.xVar))
        
    model.route_flight_constraint = ConstraintList()
    for i in range(n):
        model.route_flight_constraint.add(model.xVar[i] >= sum(model.fVar[j] for j in stringFlight[i]
                                                               if j not in multipleFlight)/len(stringFlight[i]))
        
    model.multiple_flight_constraint = ConstraintList()
    for key, value in flightString.items():
        if key in multipleFlight:
            model.multiple_flight_constraint.add(model.fVar[key] <= sum(model.xVar[v] for v in value))

    model.flight_string_constraint = ConstraintList()
    for item, value in flightString.items():
        model.flight_string_constraint.add(sum(model.xVar[v] for v in value) >= model.fVar[item])
    
    aircraftRouting = []
    priority = []
    penaltyProd = []
    reward = []
    

    
    priority.append(sum(model.fVar[j] * uniquePriorities[j] for j in range(len(sflights))))
    for i in range(n):
        reward.append(sum(rew[r]) * model.xVar[i] for r in reward_list)
            
        penaltyProd.append(penaltyCost * len(stagewise_routes[i]) * model.xVar[i])
        aircraftRouting.append(sum(prop_delay[i]) * model.xVar[i])
        
    model.value = Objective(expr = sum(aircraftRouting) * delayWeight - sum(priority) * priorityWeight
                            - sum(penaltyProd))
        
    result_obj = SolverFactory('mindtpy').solve(model, mip_solver='glpk', tee = True)
    sol_routes = []
    sol_delay = []
    index = []
    flt = []
    stringIndex = []
    c = 0
    for v in range(n):
        if model.xVar[v].value != 0:
            sol_routes.append(model.xVar[v].value)
            sol_delay.append(prop_delay[v])
            index.append(c)
            stringIndex.append(v)
        c+=1
        
    for f in range(len(sflights)):
        if model.fVar[f].value > 0.9:
            flt.append(sflights[f])
            
                
    return sol_routes, index, flt, sol_delay

In [10]:
origin_dict_og = {}
destination_dict_og = {}
object_dict_og = {}
object_dict_copy = {}
object_dict_dep_sort = {}
allAirports = []
allAircrafts = []
flightCount = {}
            

for i,r in df.iterrows():
    allAirports.append(r["ORIGIN"])
    allAirports.append(r["DEST"])
    if r["FLIGHTS"] not in flightCount:
        flightCount[r["FLIGHTS"]] = 1
    elif r["FLIGHTS"] in flightCount:
        flightCount[r["FLIGHTS"]]+=1
    origin_dict_og.update({i : originAirport(r["ORIGIN"], r["CRS_DEP_DATETIME"], r["DAY_OF_MONTH"])})
    destination_dict_og.update({i : destinationAirport(r["DEST"], r["CRS_ARR_DATETIME"], r["DAY_OF_MONTH"],
                                                       timedelta(days=0,hours=0,minutes=0,seconds=0), 0.0, 0)})

    if r["CRS_DEP_DATETIME"] not in object_dict_dep_sort:
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]] = []
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]].append(FlightLeg(r["ORIGIN"],r["DEST"],r["FLIGHTS"],r["DAY_OF_MONTH"],
                                                                     r["ARR_DELAY_NEW"], r["CRS_DEP_DATETIME"],r["CRS_ARR_DATETIME"]
                                                                     ,0,0.0,None,0.0, 0, None, 0, 0, 0, r["CLUSTER_ID"]
                                                                     , 0, np.inf
                                                                     ,r["Fleet"], 0.0, [], [], [], []))    
    else:
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]].append(FlightLeg(r["ORIGIN"],r["DEST"],r["FLIGHTS"],r["DAY_OF_MONTH"],
                                                                     r["ARR_DELAY_NEW"], r["CRS_DEP_DATETIME"],r["CRS_ARR_DATETIME"]
                                                                     ,0,0.0,None,0.0, 0, None, 0, 0, 0, r["CLUSTER_ID"]
                                                                     , 0, np.inf
                                                                     ,r["Fleet"], 0.0, [], [], [], []))



sorted_timestamp = sorted(object_dict_dep_sort)
allAirports = list(set(allAirports))
allAircrafts = list(set(allAircrafts))

In [11]:
# Sorting the object dictionary as per the time stamps
c = 0

for s in sorted_timestamp:
    for v in object_dict_dep_sort[s]:
        object_dict_og.update({c : v})
        object_dict_copy.update({c : v})
        c+=1

<span style="font-size: 20px; font-weight: bold;">Generating $\xi$ value for each flight leg using synthetically generated customer feedback. For each attribute of the flight leg mentioned in the paper anywhere between 40 and 100 people rate from 0 to 5 stars. Which is then averaged and normalised as mentioned in Section 3 of the paper.</span>

In [12]:
def give_priority(node_dict):
    maxRating = 0.0
    for n in range(len(node_dict)):
        
        for rr in range(random.randint(40,100)):
            node_dict[n].serviceratings.append(random.randint(0,5))

        for rr in range(random.randint(40,100)):
            node_dict[n].foodratings.append(random.randint(0,5))

        for rr in range(random.randint(40,100)):
            node_dict[n].entertainmentratings.append(random.randint(0,5))

        for rr in range(random.randint(40,100)):
            node_dict[n].comfortratings.append(random.randint(0,5))

        lnc = len(node_dict[n].comfortratings)
        lnf = len(node_dict[n].foodratings)
        lns = len(node_dict[n].serviceratings)
        lne = len(node_dict[n].entertainmentratings)

        node_dict[n].averageratings = (sum(node_dict[n].comfortratings)/lnc + sum(node_dict[n].foodratings)/lnf
                                       + sum(node_dict[n].serviceratings)/lns
                                       + sum(node_dict[n].entertainmentratings)/lne)/4
        node_dict[n].pre_averageratings = node_dict[n].averageratings
        if maxRating < node_dict[n].averageratings:
            maxRating = node_dict[n].averageratings
    
    for n in range(len(node_dict)):
        node_dict[n].averageratings = node_dict[n].averageratings/5

def set_priority(node_dict, count_dict, maxAverageRating):
    lbda = 0.7
    for n in range(len(node_dict)):
        nodeAvg = (1/node_dict[n].averageratings)
        node_dict[n].priorityIndex = lbda * (node_dict[n].cluster_id / 5)  + (1 - lbda) * nodeAvg/maxAverageRating
        
def normalize_priority(node_dict):
    maxPriority = 0.0
    for n in range(len(node_dict)):
        if maxPriority < node_dict[n].priorityIndex:
            maxPriority = node_dict[n].priorityIndex
    for n in range(len(node_dict)):
        node_dict[n].priorityIndex = node_dict[n].priorityIndex/maxPriority
        
def normalize_average_ratings(node_dict):
    maxAverageRating = 0.0
    for n in range(len(node_dict)):
        if maxAverageRating < (1/node_dict[n].averageratings):
            maxAverageRating = (1/node_dict[n].averageratings)
    return (maxAverageRating)


give_priority(object_dict_copy)
maxAverageRating = normalize_average_ratings(object_dict_copy)
set_priority(object_dict_copy, flightCount, maxAverageRating)
pi_list = []
for o in range(len(object_dict_copy)):
    pi_list.append(object_dict_copy[o].priorityIndex)

max_pi = max(pi_list)
for o in range(len(object_dict_copy)):
    object_dict_copy[o].priorityIndex = object_dict_copy[o].priorityIndex/max_pi

<span style="font-size: 20px; font-weight: bold;">Implementation of finding the first node algorithm as mentioned in Algorithm 3 in the paper as mentioned in the Supplementary Material.</span>

In [13]:
#To verify if the node is first node
def if_first_node(node, node_dict):
    
    res = False # Set default value of the node as not the first node
    c = 0
    for i in range(len(object_dict_og)):
        if i in node_dict:
        
            if node.origin == node_dict[i].dest:
                # Check if the departure time of the node is the least among all the nodes
                if node.dep_datetime > node_dict[i].arr_datetime:
                    if node.day >= node_dict[i].day:
        #                 if node.origin == node_dict[i].dest:
                        c += 1
                        break
                
    if c == 0: # Even if 1 node found satisfying the above conditions
        res = True
        node.processed = 1
        node.pointer = 'dummy_start'
        node.sum_priority_idx = node.cluster_id
        
    return res

In [14]:
first_node_origin = []
first_node_list = []
for i in range(len(object_dict_og)):
    if i in object_dict_copy:
        res = if_first_node(object_dict_copy[i], object_dict_copy)
        if res:
            object_dict_copy[i].elapsed_time = (object_dict_copy[i].arr_datetime - object_dict_copy[i].dep_datetime).total_seconds()
            first_node_origin.append(object_dict_copy[i].origin)
            first_node_list.append(i)

In [15]:
final_string_list = []
final_string_str = []
final_priority_list = []
f_assigned = []

<span style="font-size: 20px; font-weight: bold;">Initialising the Q Table and Reward Table for RL Framework Implementation.</span>

In [16]:
def qTableInitialize(nodes):
    
    qtcalc = {}
    rcalc = {}
    
    cc = 0
    allKeys = nodes.keys()
    for n1 in allKeys:
        for n2 in allKeys:
            if nodes[n1].day <= nodes[n2].day:
                if nodes[n1].dest == nodes[n2].origin:
                    if nodes[n1].arr_datetime < nodes[n2].dep_datetime:
                        currTimeElapsed = nodes[n1].elapsed_time
                        timeElapsed = currTimeElapsed + nodes[n2].elapsed_time
                        if check_time_elapsed_validity_seconds(timeElapsed):
                            mean_turnaround_time = 0.0
                            slack_time = calculate_slack_time_seconds(nodes[n2].dep_datetime ,nodes[n1].arr_datetime
                                                                      , nodes[n1].day, nodes[n2].day
                                                                      , mean_turnaround_time)
                            if check_slack_time_seconds_validity(slack_time):
                                qtcalc.update({cc:QTable(nodes[n1].origin, nodes[n1].dest, nodes[n1].arr_datetime
                                                         , nodes[n1].dep_datetime, nodes[n2].origin, nodes[n2].dest
                                                         , nodes[n2].arr_datetime, nodes[n2].dep_datetime
                                                         , -np.infty, 0.0, slack_time)})

                                rcalc.update({cc:Reward(nodes[n1].origin, nodes[n1].dest, nodes[n1].arr_datetime
                                                        , nodes[n1].dep_datetime, nodes[n2].origin, nodes[n2].dest
                                                        , nodes[n2].arr_datetime, nodes[n2].dep_datetime
                                                        , -np.infty, 0)})

                                cc+=1
                                        
            
    return qtcalc, rcalc

<span style="font-size: 20px; font-weight: bold;">Reward Calculation using the formula discussed in Section 6 of the paper. Based on the cluster uncertainty in delay is captured as discussed in Section 7.2.2 of the paper. Control parameters $\gamma_1$ and $\gamma_2$ introduced in the reward calculation which can be altered as per the user requirement as discussed in Section 7.2.3 of the paper. Based on the number of clusters, distribution to capture delay uncertainty is enabled. The same can be found in the delay generation code.</span>

In [17]:
def getReward(currNode, gamma1, gamma2, selectedLink, num_clusters):
    s = 0.0

    if num_clusters == 3:
        if currNode.cluster_id == 1:
            shape, scale = 15, 100.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 2:
            shape, scale = 21., 100.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 3:
            shape, scale = 27., 100.
            s = np.random.gamma(shape, scale, 1500)

    if num_clusters == 5:
        if currNode.cluster_id == 1:
            shape, scale = 2., 150.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 2:
            shape, scale = 45., 20.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 3:
            shape, scale = 15, 100.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 4:
            shape, scale = 21., 100.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 5:
            shape, scale = 27., 100.
            s = np.random.gamma(shape, scale, 1500)

    if num_clusters == 7:
        if currNode.cluster_id == 1:
            shape, scale = 2., 150.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 2:
            shape, scale = 45., 20.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 3:
            shape, scale = 45., 20.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 4:
            shape, scale = 15, 100.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 5:
            shape, scale = 15, 100.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 6:
            shape, scale = 21., 100.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 7:
            shape, scale = 27., 100.
            s = np.random.gamma(shape, scale, 1500)

    if num_clusters == 9:
        if currNode.cluster_id == 1:
            shape, scale = 2., 150.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 2:
            shape, scale = 2., 150.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 3:
            shape, scale = 45., 20.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 4:
            shape, scale = 45., 20.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 5:
            shape, scale = 15, 100.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 6:
            shape, scale = 15, 100.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 7:
            shape, scale = 15, 100.
            s = np.random.gamma(shape, scale, 1500)
            
        if currNode.cluster_id == 8:
            shape, scale = 21., 100.
            s = np.random.gamma(shape, scale, 1500)
        
        if currNode.cluster_id == 9:
            shape, scale = 27., 100.
            s = np.random.gamma(shape, scale, 1500)
        
    random.shuffle(s)
    ad = random.choice(s)
    
    return (-(gamma1 * ad)/3000 + gamma2 *(selectedLink.cluster_id)/5, ad)

<span style="font-size: 20px; font-weight: bold;">Implementation of get all feasible links algorithm as mentioned in Algorithm 7 as mentioned in the Supplementary material of the paper.</span>

In [ ]:
def getAllFeasibleLinks(currNode, nodes):
    
    feasibleLinks = {}
    
    allKeys = nodes.keys()
    for k in allKeys:
        if nodes[k].processed == 0:
            if currNode.day <= nodes[k].day:
                if currNode.dest == nodes[k].origin:
                    if currNode.arr_datetime < nodes[k].dep_datetime:
                        currTimeElapsed = currNode.elapsed_time
                        timeElapsed = currTimeElapsed + nodes[k].elapsed_time
                        if check_time_elapsed_validity_seconds(timeElapsed):
                            mean_turnaround_time = 0
                            slack_time = calculate_slack_time_seconds(nodes[k].dep_datetime ,currNode.arr_datetime
                                                                      , currNode.day, nodes[k].day
                                                                      , mean_turnaround_time)
                            if check_slack_time_seconds_validity(slack_time):
                                feasibleLinks.update({k:nodes[k]})
                                
    return feasibleLinks

<span style="font-size: 20px; font-weight: bold;">Random selection of start node and link for initial selection in the simulation. Calculation of learning rate $\alpha$ as mentioned in equation 15 of the paper</span>

In [19]:
def selectStartNode(processedNodeList):
    idx = random.randint(0, len(processedNodeList)-1)
    return idx

def selectLink(feasibleLinks):
    allKeys = feasibleLinks.keys()
    idx = random.randint(0, len(allKeys)-1)
    allKeys = list(allKeys)
    
    return allKeys[idx], feasibleLinks[allKeys[idx]]

def getAlpha(round_idx, alpha_0):
    if round_idx > 100:
        alpha = 1./np.ceil( (1/alpha_0) + ((round_idx-100.)/10) )
    else:
        alpha = alpha_0
    return alpha 

<span style="font-size: 20px; font-weight: bold;">Function to find the best link based on the Q values computed and stored in the Q Table to be used when with 1 - $\epsilon$ probability best link is to be selected.</span>

In [20]:
def find_best_link(startNode, feasibleLinks, nodes, qtcalc, previousLink, explore):
    
    allKeys = feasibleLinks.keys()
    valDict = {}
    bestLink = None
    bestIdx = -99
    
    if explore == 0:
        for k in list(allKeys):
            for q in qtcalc:
                if (feasibleLinks[k].origin == qtcalc[q].origin2 and feasibleLinks[k].dest == qtcalc[q].dest2
                    and feasibleLinks[k].arr_datetime == qtcalc[q].arrival2 and feasibleLinks[k].dep_datetime
                    == qtcalc[q].departure2 and startNode.origin == qtcalc[q].origin1 and startNode.dest
                    == qtcalc[q].dest1 and startNode.arr_datetime == qtcalc[q].arrival1 and startNode.dep_datetime
                    == qtcalc[q].departure1):
                    if k not in valDict:
                        valDict[k] = qtcalc[q].val

        bestVal = -99
        bestIdx = -99

        for k in list(allKeys):
            if k in valDict:
                if valDict[k] > bestVal:
                    bestVal = valDict[k]
                    bestIdx = k
                    
        if bestIdx != -99:
            bestLink = feasibleLinks[bestIdx]
        
    if explore == 1:
        for k in list(allKeys):
            for q in qtcalc:
                if (feasibleLinks[k].origin == qtcalc[q].origin2 and feasibleLinks[k].dest == qtcalc[q].dest2
                    and feasibleLinks[k].arr_datetime == qtcalc[q].arrival2 and feasibleLinks[k].dep_datetime
                    == qtcalc[q].departure2 and startNode.origin == qtcalc[q].origin1 and startNode.dest
                    == qtcalc[q].dest1 and startNode.arr_datetime == qtcalc[q].arrival1 and startNode.dep_datetime
                    == qtcalc[q].departure1):
                    
                    if k not in valDict:
                        valDict[k] = qtcalc[q].val
                    

        bestVal = -99
        bestIdx = -99
        allValues = []

        for k in list(allKeys):
            if k in valDict:
                allValues.append(valDict[k])
        
        if len(allValues) > 1:
            maxVal = max(allValues)
            allK = list(allKeys)
            allK.remove(allK[allValues.index(maxVal)])
            allValues.remove(maxVal)
            rand_idx = random.randrange(len(allValues))
            bestVal = allValues[rand_idx]
    #         bestVal = random.choice(allValues)
            bestIdx = allK[allValues.index(bestVal)]
            bestLink = feasibleLinks[bestIdx]
            
        else:
            maxVal = max(allValues)
            allK = list(allKeys)
            rand_idx = random.randrange(len(allValues))
            bestVal = allValues[rand_idx]
    #         bestVal = random.choice(allValues)
            bestIdx = allK[allValues.index(bestVal)]
            bestLink = feasibleLinks[bestIdx]
            
    
    return bestIdx, bestLink

<span style="font-size: 20px; font-weight: bold;">Implementation of Q Table computation framework to be used in Algorithm 6 as mentioned in the Supplementary material of the paper.</span>

In [21]:
def stringConstruction_RL(processedNodeList, nodes, rcalc, qtcalc, gamma1, gamma2, num_clusters):
    
    alpha_0 = 0.5
    gamma = 0.5
    eps = 0.4
    newProcessedNodeList = []
    res = False
#     processedKeys = 
    
    for j in range(len(processedNodeList)):
        startNodeIdx = processedNodeList[j]
        startNode = nodes[startNodeIdx]
        feasibleLinks = getAllFeasibleLinks(startNode, nodes)
        print("Evaluating for node: ", startNodeIdx)
        if len(feasibleLinks) > 0:
            
            previousLinkIdx, previousLink = selectLink(feasibleLinks)
            selectedLink = None
            for i in range(50):
                rnd = random.uniform(0,1)
                
                if rnd >= eps: #Get the best arm and not a random arm
                    if i == 0:
                        selectedLink = previousLink
                        selectedIdx = previousLinkIdx
                        if selectedLink != None:
                            selectedLink.processed = 1
                            
                    else:
                        selectedIdx, selectedLink = find_best_link(startNode, feasibleLinks, nodes, qtcalc
                                                                   , previousLink, 0)
                        if selectedLink != None:
                            selectedLink.processed = 1
                            
                if rnd < eps: #Get a randomly selected arm which is not the best one
                    selectedIdx, selectedLink = find_best_link(startNode, feasibleLinks, nodes, qtcalc, previousLink, 1)
                    selectedLink.processed = 1
                    
                if selectedLink != None:
                    for r in rcalc:
                        if (rcalc[r].origin1 == startNode.origin and rcalc[r].dest1 == startNode.dest
                            and rcalc[r].arrival1 == startNode.arr_datetime and rcalc[r].departure1
                            == startNode.dep_datetime
                            and rcalc[r].origin2 == selectedLink.origin and rcalc[r].dest2 == selectedLink.dest
                            and rcalc[r].arrival2 == selectedLink.arr_datetime
                            and rcalc[r].departure2 == selectedLink.dep_datetime):
                            #Place an if condition to consider - infinity rewards 
                            if rcalc[r].reward == -np.infty:
#                                 print(getReward(startNode, gamma1, gamma2, selectedLink))
                                rcalc[r].reward, arrDelay = getReward(startNode, gamma1, gamma2, selectedLink, num_clusters)
                                rcalc[r].number += 1
                                selectedLink.total_propagated_delay += calculate_propagated_delay_seconds(startNode.total_propagated_delay,arrDelay,qtcalc[r].slack_time)
                            else:
                                rew, arrDelay = getReward(startNode, gamma1, gamma2, selectedLink, num_clusters)
                                rcalc[r].reward += rew
                                rcalc[r].number += 1
                                selectedLink.total_propagated_delay += calculate_propagated_delay_seconds(startNode.total_propagated_delay,arrDelay,qtcalc[r].slack_time)

                            rAvg = rcalc[r].reward/rcalc[r].number
                            selectedLink.total_propagated_delay = selectedLink.total_propagated_delay/rcalc[r].number
                            look_ahead_best = 0
                            look_ahead_values = []

                            for rr in rcalc:
                                if (rcalc[rr].origin1 == selectedLink.origin and rcalc[rr].dest1 == selectedLink.dest
                                    and rcalc[rr].arrival1 == selectedLink.arr_datetime
                                    and rcalc[rr].departure1 == selectedLink.dep_datetime):
                                    look_ahead_values.append(rcalc[rr].reward)

                            look_ahead_best = -np.infty
                            if len(look_ahead_values) > 0:
                                look_ahead_best = max(look_ahead_values)

                            #If condition to handle -infinity values in qtcalc
                            if look_ahead_best > -np.infty:
                                if qtcalc[r].val == -np.infty:
                                    qtcalc[r].val = getAlpha(i, alpha_0) * (rAvg + gamma * look_ahead_best
                                                                            - qtcalc[r].val)
                                    newProcessedNodeList.append(selectedIdx)
                                    res = True

                                else:
                                    print(qtcalc[r].val, getAlpha(i, alpha_0),(rAvg + gamma * look_ahead_best
                                                                               - qtcalc[r].val))
                                    qtcalc[r].val = qtcalc[r].val + getAlpha(i, alpha_0) * (rAvg + gamma * look_ahead_best
                                                                                            - qtcalc[r].val)
                                    newProcessedNodeList.append(selectedIdx)
                                    res = True

                            else:
                                if qtcalc[r].val == -np.infty:
                                    qtcalc[r].val = getAlpha(i, alpha_0) * (rAvg + gamma * 0.0)
                                else:
                                    qtcalc[r].val = getAlpha(i, alpha_0) * (rAvg + gamma * 0.0 - qtcalc[r].val)
                                newProcessedNodeList.append(selectedIdx)
                                res = True
                            break
                        
    return newProcessedNodeList

<span style="font-size: 20px; font-weight: bold;">Implementation of String construction mechanism based on the Q Table computed as mentioned in Algorithm 8 in the paper as mentioned in the Supplementary Material.</span>

In [22]:
def pathConstruction_RL(nodes, qtcalc, first_nodes_list, gamma1, gamma2, num_clusters):
    
    route_list = []
    route_list_str = []
    priority_list = []
    time_list_str = []
    delay_list = []
    priorityDict = {}
    
    for f in first_nodes_list:
        res = True
        dummy_route = []
        dummy_route_str = []
        dummy_priority_list = []
        dummy_time_str = []
        dummy_delay_list = []
            
        org = nodes[f].origin
        dst = nodes[f].dest
        arr = nodes[f].arr_datetime
        dep = nodes[f].dep_datetime
        
        subroute_str = str(nodes[f].origin)+'-'+str(nodes[f].dest)
        subroute_time_str = str(nodes[f].origin) + '-' + str(nodes[f].dep_datetime)+'-'+str(nodes[f].dest)  + '-' + str(nodes[f].arr_datetime)
            
        dummy_route.append(f)
        dummy_route_str.append(subroute_str)
        dummy_priority_list.append(nodes[f].priorityIndex)
        dummy_time_str.append(subroute_time_str)
        dummy_delay_list.append(0.0)
        if f not in priorityDict:
            priorityDict[f] = nodes[f].priorityIndex
            
        while (res == True):
            valList = []
            valList.append(-99)
            while len(valList) != 0:
                vlist = []
                ilist = []
                for q in range(len(qtcalc)):
                    if (qtcalc[q].origin1 == org and qtcalc[q].dest1 == dst and qtcalc[q].arrival1 == arr
                        and qtcalc[q].departure1 == dep):
                        if qtcalc[q].val > -np.infty:
                            vlist.append(qtcalc[q].val)
                            ilist.append(q)
                            
                
                if len(vlist) > 0:
                    qidx = ilist[vlist.index(max(vlist))] # change reward to -rewards
                    for n in nodes:
                        if(nodes[n].origin == qtcalc[qidx].origin2 and nodes[n].dest == qtcalc[qidx].dest2
                           and nodes[n].arr_datetime == qtcalc[qidx].arrival2
                           and nodes[n].dep_datetime == qtcalc[qidx].departure2):
                            subroute_str = str(nodes[n].origin)+'-'+str(nodes[n].dest)
                            subroute_time_str = str(nodes[n].origin) + '-' + str(nodes[n].dep_datetime)+'-'+str(nodes[n].dest)  + '-' + str(nodes[n].arr_datetime)
                            
                            temp, arrDel = getReward(nodes[dummy_route[len(dummy_route)-1]], gamma1, gamma2, nodes[n], num_clusters)
                            nodes[n].arr_delay = arrDel
                            nodes[n].total_propagated_delay = propDelaySeconds_RL(nodes[dummy_route[len(dummy_route)-1]],nodes[n])
                            dummy_route.append(n)
                            dummy_route_str.append(subroute_str)
                            dummy_priority_list.append(nodes[n].priorityIndex)
                            dummy_time_str.append(subroute_time_str)
                            dummy_delay_list.append(nodes[n].total_propagated_delay)
                            if n not in priorityDict:
                                priorityDict[n] = nodes[n].priorityIndex
                            
                    org = qtcalc[qidx].origin2
                    dst = qtcalc[qidx].dest2
                    arr = qtcalc[qidx].arrival2
                    dep = qtcalc[qidx].departure2
                            
                if len(vlist) == 0:
                    valList = []
                    res = False
                    
        if len(dummy_route) > 1:
            route_list.append(dummy_route)
            route_list_str.append(dummy_route_str)
            priority_list.append(dummy_priority_list)
            time_list_str.append(dummy_time_str)
            delay_list.append(dummy_delay_list)

    return (route_list, route_list_str, priority_list, time_list_str, priorityDict, delay_list)

In [23]:
#Do this in the path constructor itslef, no need for another function
def rewardListConstructor_RL(pathList, nodes, qtcalc):
    
    reward_list = []
    
    for pl in pathList:
        counter = 0
        dummy_reward_list = []
        for p in pl:
            for q in qtcalc:
                if counter+1 < len(pl):
                    if (nodes[p].origin == qtcalc[q].origin1 and nodes[p].dest == qtcalc[q].dest1
                        and nodes[p].arr_datetime == qtcalc[q].arrival1
                        and nodes[p].dep_datetime == qtcalc[q].departure1 and nodes[pl[counter+1]].origin
                        == qtcalc[q].origin2 and nodes[pl[counter+1]].dest == qtcalc[q].dest2 and
                        nodes[pl[counter+1]].arr_datetime == qtcalc[q].arrival2
                        and nodes[pl[counter+1]].dep_datetime == qtcalc[q].departure2):
                        dummy_reward_list.append(qtcalc[q].val)
        
        reward_list.append(dummy_reward_list)
    
    return reward_list

In [24]:
def removeLinks(rcalc, qtcalc, nodes, deletedNodes, maxComb):
    
    idxToRemove = []
    for d in deletedNodes:
        for q in range(maxComb):
            if q in qtcalc:
                if (qtcalc[q].origin1 == nodes[d].origin and qtcalc[q].dest1 == nodes[d].dest
                    and qtcalc[q].arrival1 == nodes[d].arr_datetime
                    and qtcalc[q].departure1 == nodes[d].dep_datetime):
                    if q not in idxToRemove:
                        idxToRemove.append(q)
                    
            if q in qtcalc:
                if (qtcalc[q].origin2 == nodes[d].origin and qtcalc[q].dest2 == nodes[d].dest
                    and qtcalc[q].arrival2 == nodes[d].arr_datetime
                    and qtcalc[q].departure2 == nodes[d].dep_datetime):
                    if q not in idxToRemove:
                        idxToRemove.append(q)
                        
    for i in idxToRemove:
        del qtcalc[i]
        del rcalc[i]
    
    return qtcalc

<span style="font-size: 20px; font-weight: bold;">Implementation of RL General Framework as mentioned in Algorithm 6 in the paper as mentioned in the Supplementary Material.</span>

In [25]:
t0 = time.process_time()
fl_list = []
finalRevenue = []
finalSpillCost = []
finalOperatingCost = []
finalDelay = []
priorityWeight = 1
delayWeight = 1
priority_number_global = int(0.25 * len(object_dict_og))
priority_number = priority_number_global
penaltyCost = 1000
gamma2 = 1
gamma1 = 1
sol = []
s_delay = []
flightAssigned = []
sol.append(-99)
tsearch = []
topt = []
tpath = []
tlink = []
stringLen = []
itr = 1
dispPriorityList = []
dispPathList = []

while len(sol) != 0:
    ts1 = time.process_time()
    qtcalc, rcalc = qTableInitialize(object_dict_copy)
    first_node_tuple = []
    first_node_list = []
    for i in range(len(object_dict_og)):
        if i in object_dict_copy:
            res = if_first_node(object_dict_copy[i], object_dict_copy)
            if res:
                first_node_tuple.append((i, object_dict_copy[i].origin))
                first_node_list.append(i)
    
    print("*****************************")
    print("printing first node")
    print(first_node_list)
    nextProcessedList = []
    nextProcessedList = first_node_list
    
    while len(nextProcessedList) != 0:
        nextProcessedList = stringConstruction_RL(nextProcessedList, object_dict_copy, rcalc, qtcalc, gamma1, gamma2, num_clusters)
        nextProcessedList = list(OrderedDict.fromkeys(nextProcessedList))
        print("++++++++++++++++++++++++++")
        print("printing next processed list")
        print(nextProcessedList)
        print("++++++++++++++++++++++++++")
        
    path_list, path_list_str, priority_list, time_list, priorityDict, prop_delay_list = pathConstruction_RL(object_dict_copy, qtcalc, first_node_list, gamma1, gamma2, num_clusters)
    
    print("*******************")
    print("printing path list and attributes")
    print(path_list)
    print(path_list_str)
    print("*******************")

    
#     prop_delay_list = []
    sflights = []
    assignedFlights = []
    uniquePriorities = []
    
    for pl in path_list:
        for p in pl:
            if p not in sflights:
                sflights.append(p)
            if p not in assignedFlights:
                if object_dict_copy[p].priorityAssigned == 1:
                    assignedFlights.append(p)
                    
    for s in sflights:
        uniquePriorities.append(priorityDict[s])
    
    orgCounter = []
    if len(path_list) > 0:
        to1 = time.process_time()
        flightsFed = []
        for pl in path_list:
            for p in pl:
                if p not in flightsFed:
                    flightsFed.append(p)
        stringLen.append(len(path_list))
        reward_list = rewardListConstructor_RL(path_list, object_dict_copy, qtcalc)
        print("Printing other attributes")
        print(path_list)
        print(prop_delay_list)
        print(sflights)
        print(reward_list)
        sol, idx, flt, sdelay, = route_optimizer(path_list, prop_delay_list, path_list_str, priority_list
                                                 , priority_number, sflights, assignedFlights, itr, penaltyCost
                                                 , priorityWeight, delayWeight, uniquePriorities, reward_list)
        
        print("+++++++++++++++++++++++")
        print("printing solution from optimizer")
        print(sol)
        print("+++++++++++++++++++++++")

        for i in idx:
            dispPriorityList.append(priority_list[i])
            dispPathList.append(path_list[i])
        itr+=1
        finalDelay.append(sdelay)
        if len(flt) > 0:
            for fll in flt: 
                f_assigned.append(fll)

        for s in sdelay:
            s_delay.append(s)

        sol_path = []
        flightsAfterselection = []
        for i in idx:
            sol_path.append(time_list[i])
            for pl in path_list[i]:
                if pl not in flightsAfterselection:
                    flightsAfterselection.append(pl)



        ff = 0
        while ff < len(flt):
            fl = flt[ff]
            if object_dict_copy[fl].priorityAssigned != 1:
                object_dict_copy[fl].priorityAssigned = 1
                ff+=1
            elif object_dict_copy[fl].priorityAssigned == 1:
                flt.remove(fl)
            fl_list.append(fl)


        priority_number -= len(flt)
        for ob in range(len(object_dict_og)):
            if ob in object_dict_copy:
                if object_dict_copy[ob].priorityAssigned == 1:
                    fla = str(object_dict_copy[ob].origin) + str("-") + str(object_dict_copy[ob].dest + str("-") + str(object_dict_copy[ob].arr_datetime) + str("-") + str(object_dict_copy[ob].dep_datetime))
                    flightAssigned.append(fla)

        if len(sol) > 0:
            for i in idx:
                for j in path_list[i]:
                    if j in object_dict_copy:
                        if j in first_node_list:
                            orgCounter.append((object_dict_copy[j].origin, object_dict_copy[path_list[i][-1]].dest))

                        del object_dict_copy[j]

    if len(path_list) == 0:
        sol = []

    if len(path_list) > 0:
        for i in idx:
            final_string_list.append(path_list[i])
            final_string_str.append(path_list_str[i])
            final_priority_list.append(priority_list[i])

    for i in range(len(object_dict_og)):
        if i in object_dict_copy:
            object_dict_copy[i].pointer = None
            object_dict_copy[i].processed = 0
            object_dict_copy[i].elapsed_time = 0.0
            object_dict_copy[i].total_propagated_delay = 0.0

    ts2 = time.process_time()
    tsearch.append(ts2 - ts1)

t1 = time.process_time()

*****************************
printing first node
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 51, 93, 96, 122, 142, 144, 157, 194, 224, 450]
Evaluating for node:  0
Evaluating for node:  1
Evaluating for node:  2
Evaluating for node:  3
Evaluating for node:  4
Evaluating for node:  5
Evaluating for node:  6
Evaluating for node:  7
Evaluating for node:  8
Evaluating for node:  9
Evaluating for node:  10
Evaluating for node:  11
Evaluating for node:  12
Evaluating for node:  13
Evaluating for node:  14
Evaluating for node:  15
Evaluating for node:  16
Evaluating for node:  17
Evaluating for node:  18
Evaluating for node:  19
Evaluating for node:  20
Evaluating for node:  21
Evaluating for node:  22
Evaluating for node:  23
Evaluating for node:  25
Evaluating for node:  26
Evaluating for node:  27
Evaluating for node:  29
Evaluating for node:  51
Evaluating for node:  93
Evaluating for node:  96
Evaluating for node:  122
Evaluatin

Starting MindtPy version 0.1.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C894650>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C894690>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C8946D0>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: false
heuristic_nonconvex: false
init_strategy: rNLP
level_coef: 0.5
solution_limit

+++++++++++++++++++++++
printing solution from optimizer
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
+++++++++++++++++++++++
*****************************
printing first node
[24, 25, 27, 28, 31, 34, 35, 36, 37, 38, 40, 42, 44, 45, 46, 47, 48, 51, 52, 60, 65, 68, 69, 71, 72, 74, 78, 80, 84, 87, 88, 94, 97, 100, 105, 113, 128, 135, 157]
Evaluating for node:  24
Evaluating for node:  25
Evaluating for node:  27
Evaluating for node:  28
Evaluating for node:  31
Evaluating for node:  34
Evaluating for node:  35
Evaluating for node:  36
Evaluating for node:  37
Evaluating for node:  38
Evaluating for node:  40
Evaluating for node:  42
Evaluating for node:  44
Evaluating for node:  45
Evaluating for node:  46
Evaluating for node:  47
Evaluating for node:  48
Evaluating for node:  51
Evaluating for node:  52
Evaluating for node:  60
Evaluating for node:  65
Evaluating

Starting MindtPy version 0.1.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C894650>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C894690>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C8946D0>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)>
logging_level: 20
integer_to_binary: false
add_no_good_cuts: false
use_tabu_list: false
single_tree: false
solution_pool: false
num_solution_iteration: 5
cycling_check: true
feasibility_norm: L_infinity
differentiate_mode: reverse_symbolic
use_mcpp: false
calculate_dual_at_solution: false
use_fbbt: false
use_dual_bound: true
partition_obj_nonlinear_terms: true
quadratic_strategy: 0
move_objective: false
add_cuts_at_incumbent: false
heuristic_nonconvex: false
init_strategy: rNLP
level_coef: 0.5
solution_limit

Printing other attributes
[[24, 744, 884], [25, 92], [27, 58, 1022], [28, 174], [31, 976], [34, 388], [36, 901], [37, 236], [38, 102], [40, 1045], [42, 533], [44, 1029], [45, 407], [46, 163], [47, 178], [48, 327], [52, 161], [60, 231], [65, 356, 502], [68, 361], [69, 350], [71, 1002], [72, 642], [74, 247], [80, 523], [84, 376], [87, 197, 290], [88, 713], [94, 875], [97, 456], [100, 538], [105, 229], [113, 769], [128, 1013], [135, 794], [157, 653]]
[[0.0, 0, 642.391707950035], [0.0, 0], [0.0, 0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 830.1105655395252], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0, 348.8911191352954], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 795.1708618847792], [0.0, 0], [0.0, 0], [0.0, 0, 75.36518355054295], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0], [0.0, 0]]
[24, 744, 884, 25, 92, 27, 58, 1022, 28, 174, 31, 976, 34, 388, 36, 901, 37, 236, 38, 102, 40, 1

...

Evaluating for node:  732
Evaluating for node:  733
Evaluating for node:  737
Evaluating for node:  739
Evaluating for node:  745
Evaluating for node:  746
Evaluating for node:  754
Evaluating for node:  767
Evaluating for node:  772
Evaluating for node:  781
Evaluating for node:  788
Evaluating for node:  793
Evaluating for node:  795
Evaluating for node:  807
Evaluating for node:  821
Evaluating for node:  824
Evaluating for node:  827
Evaluating for node:  829
Evaluating for node:  835
Evaluating for node:  836
Evaluating for node:  838
Evaluating for node:  845
Evaluating for node:  849
Evaluating for node:  856
Evaluating for node:  859
Evaluating for node:  866
Evaluating for node:  868
Evaluating for node:  874
Evaluating for node:  880
Evaluating for node:  888
Evaluating for node:  893
Evaluating for node:  916
Evaluating for node:  918
Evaluating for node:  932
Evaluating for node:  933
Evaluating for node:  940
Evaluating for node:  961
Evaluating for node:  966
Evaluating f

         -       Relaxed NLP            -60000            inf         -60000      nan%      0.08
         1              MILP            -60000            inf         -60000      nan%      0.12
*        1         Fixed NLP            -60000         -60000         -60000    -0.00%      0.18
MindtPy exiting on bound convergence. Absolute gap: -3.584864316508174e-08 <= absolute tolerance: 0.0001 

 Primal integral          :    0.0000 
 Dual integral            :    0.0000 
 Primal-dual gap integral :    0.0000 
Starting MindtPy version 0.1.0 using OA algorithm
iteration_limit: 50
stalling_limit: 15
time_limit: 600
strategy: OA
add_regularization: None
call_after_main_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C894650>
call_after_subproblem_solve: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C894690>
call_after_subproblem_feasible: <pyomo.contrib.gdpopt.util._DoNothing object at 0x000001712C8946D0>
tee: true
logger: <Logger pyomo.contrib.mindtpy (INFO)

+++++++++++++++++++++++
printing solution from optimizer
[1.0, 1.0, 1.0]
+++++++++++++++++++++++
*****************************
printing first node
[73, 286, 353, 409, 445, 447, 553, 561, 585, 617, 622, 655, 666, 699, 712, 715, 719, 720, 732, 733, 737, 739, 745, 746, 754, 767, 772, 781, 788, 793, 795, 807, 821, 824, 827, 829, 835, 836, 838, 845, 849, 856, 859, 866, 868, 874, 880, 888, 893, 916, 918, 932, 933, 940, 961, 966, 977, 980, 998, 1009, 1010, 1026, 1031, 1037]
Evaluating for node:  73
Evaluating for node:  286
Evaluating for node:  353
Evaluating for node:  409
Evaluating for node:  445
Evaluating for node:  447
Evaluating for node:  553
Evaluating for node:  561
Evaluating for node:  585
Evaluating for node:  617
Evaluating for node:  622
Evaluating for node:  655
Evaluating for node:  666
Evaluating for node:  699
Evaluating for node:  712
Evaluating for node:  715
Evaluating for node:  719
Evaluating for node:  720
Evaluating for node:  732
Evaluating for node:  733
Evaluatin

         -       Relaxed NLP            -20000            inf         -20000      nan%      0.06
         1              MILP            -20000            inf         -20000      nan%      0.09
*        1         Fixed NLP            -20000         -20000         -20000    -0.00%      0.13
MindtPy exiting on bound convergence. Absolute gap: -3.82351572625339e-09 <= absolute tolerance: 0.0001 

 Primal integral          :    0.0000 
 Dual integral            :    0.0000 
 Primal-dual gap integral :    0.0000 


+++++++++++++++++++++++
printing solution from optimizer
[1.0]
+++++++++++++++++++++++
*****************************
printing first node
[73, 286, 353, 409, 445, 447, 553, 561, 585, 617, 622, 655, 666, 699, 712, 715, 719, 720, 732, 733, 737, 745, 746, 754, 767, 772, 781, 788, 793, 795, 807, 821, 824, 827, 829, 835, 836, 838, 845, 849, 856, 859, 866, 868, 874, 880, 888, 893, 916, 918, 932, 933, 940, 961, 966, 977, 980, 998, 1009, 1010, 1026, 1031, 1037]
Evaluating for node:  73
Evaluating for node:  286
Evaluating for node:  353
Evaluating for node:  409
Evaluating for node:  445
Evaluating for node:  447
Evaluating for node:  553
Evaluating for node:  561
Evaluating for node:  585
Evaluating for node:  617
Evaluating for node:  622
Evaluating for node:  655
Evaluating for node:  666
Evaluating for node:  699
Evaluating for node:  712
Evaluating for node:  715
Evaluating for node:  719
Evaluating for node:  720
Evaluating for node:  732
Evaluating for node:  733
Evaluating for node:  73

In [26]:
print(final_string_list)
print(final_string_str)
print(s_delay)
print(fl_list)
times = []
priority_list = []
for fs in final_string_list:
    temp = []
    pr_temp = []
    for f in fs:
        start_dt = object_dict_og[f].dep_datetime
        end_dt = object_dict_og[f].arr_datetime

        start_time = start_dt.strftime("%H:%M")
        end_time = end_dt.strftime("%H:%M")
        temp.append(str(start_time) + str("-") + str(end_time))
        pr_temp.append(object_dict_og[f].priorityIndex)
    times.append(temp)
    priority_list.append(pr_temp)
print(times)
tups = []
counter = 0
for fs in final_string_str:
    temp = []
    cc = 0
    for f in fs:
        temp.append((f.split('-')[0], f.split('-')[1], times[counter][cc].split("-")[0]
                     , times[counter][cc].split("-")[1]
                     , object_dict_og[final_string_list[counter][cc]].arr_delay))
        cc+=1
    counter+=1
    tups.append(temp)
print(tups)
print(priority_list)

extcounter = 0
delay_clusterList = []
delay_obs = []

for sd in s_delay:
    intcounter = 0
    for s in sd:
        if s > 0:
            idx = final_string_list[extcounter][intcounter]
            delay_clusterList.append(object_dict_og[idx].cluster_id)
            delay_obs.append(s)
        intcounter += 1
    extcounter += 1
            
print(delay_clusterList)
print(delay_obs)

[[0, 167, 974], [1, 33, 1011], [2, 496, 1016], [3, 209], [4, 203], [5, 232, 367], [6, 30], [7, 283, 934], [8, 762], [9, 554], [10, 430, 894], [11, 154], [12, 1044], [13, 32], [14, 576], [15, 692], [16, 952], [17, 852], [18, 779, 876], [19, 996], [20, 249], [21, 1004], [22, 680], [23, 644], [26, 195], [29, 63], [93, 228], [96, 621], [122, 886], [142, 1048], [144, 897], [194, 323], [224, 832], [450, 522], [24, 744, 884], [25, 92], [27, 58, 1022], [28, 174], [31, 976], [34, 388], [36, 901], [37, 236], [38, 102], [40, 1045], [42, 533], [44, 1029], [45, 407], [46, 163], [47, 178], [48, 327], [52, 161], [60, 231], [65, 356, 502], [68, 361], [69, 350], [71, 1002], [72, 642], [74, 247], [80, 523], [84, 376], [87, 197, 290], [88, 713], [94, 875], [97, 456], [100, 538], [105, 229], [113, 769], [128, 1013], [135, 794], [157, 653], [35, 101, 751], [39, 160], [41, 306], [43, 1041], [49, 171, 334], [50, 338], [53, 347], [54, 312], [55, 581], [56, 679], [59, 597], [62, 252], [64, 413], [66, 196], [67

In [29]:
print("Residual Flight legs count: ", len(object_dict_copy))

Residual Flight legs count:  65


In [30]:
print("Total Delay: ", sum(delay_obs)/60)

Total Delay:  1107.7330138660009
